In [ ]:
%%html
<a href="https://colab.research.google.com/github/tejnica/creatoria-lite/blob/main/run_demo.py" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# 📌 Creatoria-lite: minimal open-source design optimizer
#
# YAML → QUBO → Simulated Annealing → CSV → Pareto
#
# - 🔧 Binary options: `cell_thickness`, `fin_height`
# - 🎯 Objectives: maximize thermal conductivity (−λ), minimize mass
# - 🧊 Solver: `dwave-neal` (Simulated Annealing)
#
# Output: `results.csv`, Pareto chart `pareto.png`


In [ ]:
!pip install -q pyqubo dwave-neal pymoo matplotlib pyyaml pandas


In [ ]:
from pyqubo import Binary
import pandas as pd
import yaml
from neal import SimulatedAnnealingSampler
import matplotlib.pyplot as plt

# Build QUBO
def build_qubo(path='contradiction.yaml'):
    spec = yaml.safe_load(open(path))
    H = 0
    binaries = {}
    for name, cfg in spec["options"].items():
        x = Binary(name)
        binaries[name] = x
        H += cfg["weight"] * x + abs(cfg["effect_lambda"]) * x
    Q, _ = H.compile().to_qubo()
    return Q, binaries, spec

# Solve QUBO
def solve(n_reads=500):
    Q, binaries, spec = build_qubo()
    sampler = SimulatedAnnealingSampler()
    response = sampler.sample_qubo(Q, num_reads=n_reads, num_sweeps=2000)

    records = []
    var_order = list(binaries.keys())

    for sample, _ in zip(response.record.sample, response.record.energy):
        sel = dict(zip(var_order, sample))
        lambda_neg = 0
        mass = 0
        for name, bit in sel.items():
            cfg = spec["options"][name]
            lambda_neg += cfg["effect_lambda"] * bit
            mass += cfg["weight"] * bit
        records.append({"lambda_neg": lambda_neg, "mass": mass})

    df = pd.DataFrame(records)
    df.to_csv("results.csv", index=False)
    return df

# Plot Pareto
def plot():
    df = pd.read_csv("results.csv")
    df = df.drop_duplicates()

    pareto = df.sort_values(['lambda_neg', 'mass'], ascending=[True, True])
    pareto_front = [pareto.iloc[0]]
    for i in range(1, len(pareto)):
        if pareto.iloc[i]['mass'] < pareto_front[-1]['mass']:
            pareto_front.append(pareto.iloc[i])
    pareto_front = pd.DataFrame(pareto_front)

    plt.figure(figsize=(6, 5))
    plt.scatter(df["lambda_neg"], df["mass"], color="orange", alpha=0.8, label="Candidates", marker="*")
    plt.plot(pareto_front["lambda_neg"], pareto_front["mass"], color="black", label="Pareto front")
    plt.xlabel("-lambda")
    plt.ylabel("mass")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("pareto.png")

# Main
df = solve()
plot()
df.drop_duplicates()


In [ ]:
from IPython.display import Image, display
display(Image("pareto.png"))


In [ ]:
with open("contradiction.yaml", "w") as f:
    f.write("""name: demo
objectives:
  - id: lambda_neg
    description: "минус теплопроводность"
  - id: mass
    description: "масса"
options:
  cell_thickness:
    bits: 1
    weight: 80
    effect_lambda: -0.9
  fin_height:
    bits: 1
    weight: 60
    effect_lambda: -0.7
""")
